In [2]:
import re
import pandas as pd
import numpy as np
import os

In [3]:
file1 = 'Data/20190819_WPS_NORM.hackathon_sample_info_updated.csv'
file2 = 'Data/20190815_WPS_NORM.hackathon_sample_info.csv'
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

common_df = df1[df1['library'].isin(df2['library'])]

In [4]:
# Evaluate machine learning model performance for patients with tumor purity < 10%
common_df = common_df[common_df['estimated.purity'] <= 0.1] 
common_df = common_df.fillna('N/A')

In [5]:
# Identify presence of oncogene
def search (test):
    if test != 'N/A':
        if re.search('PIK3CA',test) or re.search('BRAF',test) or re.search('KRAS',test) or re.search('ESR1',test):
            return ('Positive')
        elif test:
            return ('Boderline')
    else:
        return ('Negative')

In [6]:
common_df['type'] = common_df['hotspot'].apply(lambda row: search(row))

In [7]:
pos = common_df[common_df['type'] == 'Positive']
neg = common_df[common_df['type'] == 'Negative']
bod = common_df[common_df['type'] == 'Boderline']

p = pos['library'].tolist()
n = neg['library'].tolist()
b = bod['library'].tolist()

In [14]:
files = os.listdir('FeaturesResults/TestingSet/')

In [15]:
# Label the quality of the prediction (caution: arbitrary acceptable value)
def testing(row): 
    r = float(row)
    if r <= 0.1:
        return ('Concordant')
    elif r <= 0.11:
        return ('Acceptable')
    else:
        return ('Unacceptable')

In [18]:
oncogene = ['Negative']
dir_name = 'FeaturesResults/TestingSet/'
for q in oncogene:
    quality_final = [['Model','Concordant','Acceptable','Unacceptable','Percentage']]
    for file in files:
        temp = []
        filename = os.path.join(dir_name,file)
        test_df = pd.read_csv(filename)
        
        # Only select target samples
        test_df = test_df[test_df['PatientId'].isin(n)] 
        test_df['Model_quality'] = test_df['Y_fitted'].apply(lambda row: testing(row))
        temp = test_df['Model_quality'].value_counts()
        if 'Concordant' not in temp:
            temp['Concordant'] = 0
        if 'Unacceptable' not in temp:
            temp['Unacceptable'] = 0
        if 'Acceptable' not in temp:
            temp['Acceptable'] = 0
        temp['Percentage'] = (temp['Acceptable']+temp['Concordant'])/(temp['Acceptable']+temp['Concordant']+temp['Unacceptable'])
        li = [file, temp['Concordant'], temp['Acceptable'], temp['Unacceptable'], temp['Percentage']]
        quality_final.append(li)
        quality_control_train = pd.DataFrame(quality_final)
    
    quality_control_train.to_csv(f'Testing_data_prediction_quality_{q}_Patients.csv',sep='\t')